In [ ]:
#!pip install -q fast-ml

In [ ]:
import os
# os.chdir(os.path.dirname('/Users/danielmlow/Dropbox (MIT)/datum/ctl/'))
# import pickle 
from collections import defaultdict
import re
import pandas as pd
import numpy as np
import sys
import pickle
# sys.path.insert(1,'./../concept-tracker') # okay for python terminal interactive console
sys.path.insert(1,'/Users/danielmlow/Dropbox (MIT)/datum/concept-tracker') # needed for jupyter interactive panel #TODO install
from concept_tracker.utils.datetime_conversions import list_months_between_interval
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", None)



In [ ]:
import datetime
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

In [ ]:
input_dir = './../../../data/ctl/'
output_dir = './data/input/ctl/'


In [ ]:
import datetime
def generate_timestamp(format = '%y-%m-%dT%H-%M-%S'):
	ts = datetime.datetime.utcnow().strftime(format) # so you don't overwrite, and save timestamp
	return ts

# Split into train-val-test sets

### From 2016 when we have tags (from 2016-03-05)

In [ ]:
year_months = list_months_between_interval(start='2017-01-01', end='2023-01-01')
years = [2017, 2018, 2019, 2022] #removed 2020 and 2021, heavily biased by the pandemic

In [ ]:
%%time
convo_ids_all = [] 
years_all = []
for year in years:
    metadata_i = pd.read_parquet(input_dir+f'raw_data/tags_risk_{year}_dummyvars.gzip', engine='pyarrow')        
    convo_ids = metadata_i['conversation_id'].values
    convo_ids_all.append(convo_ids)
    years_all.append([year]*len(convo_ids))

    

In [ ]:
convo_ids_all = [n for i in convo_ids_all for n in i]
years_all = [n for i in years_all for n in i]


In [ ]:
len(convo_ids_all)

In [ ]:
# convo_train, years_train, convo_val, years_val, convo_test, years_test = train_valid_test_split(df, target = 'SalePrice', 
                                                                            # train_size=0.8, valid_size=0.1, test_size=0.1)


In [ ]:
convo_ids_train, convo_ids_valtest, years_train, years_valtest = train_test_split(convo_ids_all, years_all, test_size = 0.2, random_state=123)
convo_ids_val, convo_ids_test, years_val, years_test = train_test_split(convo_ids_valtest, years_val_test, test_size = 0.5, random_state=123)


In [ ]:
print('train:', len(convo_ids_train))
print('val:', len(convo_ids_val))
print('test:', len(convo_ids_test))

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
for years_set, name in zip([years_train, years_val, years_test], ['train', 'val', 'test']):
    counts = Counter(years_set)
    print(name, counts)
    # plt.bar(x=counts.keys(), height = counts.values())
    # plt.title(name)
    # plt.show()

### Obtain smaller set for concurrent validity

In [ ]:
convo_ids_train90, convo_ids_train10, years_train90, years_train10 = train_test_split(convo_ids_train, years_train, test_size = 0.1, random_state=123)


In [ ]:
print(len(convo_ids_train90), len(convo_ids_train10))

# From train_10 create train10 train, val, test pkl splits

In [ ]:
convo_ids_train10_train, convo_ids_train10_valtest, years_train10_train, years_train10_valtest = train_test_split(convo_ids_train_10, years_train10, test_size = 0.2, random_state=123)
convo_ids_train10_val, convo_ids_train10_test, years_train10_val, years_train10_test = train_test_split(convo_ids_train10_valtest, years_train10_valtest, test_size = 0.5, random_state=123)


In [ ]:
print(len(convo_ids_train10_train), len(convo_ids_train10_val),len(convo_ids_train10_test))

In [ ]:
sets = [
    [years_train, convo_ids_train], 
    [years_val, convo_ids_val], 
    [years_test, convo_ids_test], 
    [years_train90, convo_ids_train90], 
    [years_train10_train, convo_ids_train10_train],
    [years_train10_val, convo_ids_train10_val],
    [years_train10_test, convo_ids_train10_test],
    ]

In [ ]:
sets_names = [
    'train', 
    'val', 
    'test', 
    'train90', 
    'train10_train',
    'train10_val',
    'train10_test',
]


In [ ]:
input_dir

In [ ]:


os.makedirs('./data/input/train_test_split_ids/', exist_ok=True)

In [ ]:
d

In [ ]:
years = ['a','b','a','b','a','b']
convo_ids = [1,2,3,4,5,6]

d = defaultdict(list)
for A, B in zip(years, convo_ids):
    d[A].append(B)
    
d

In [ ]:
%%time 
# 19 s
run_this = False

if run_this:

    for (years, convo_ids), name in zip(sets, sets_names):
        d = defaultdict(list)
        for A, B in zip(years, convo_ids):
            d[A].append(B)

        with open(input_dir+f'train_test_split_ids/{name}.pkl', 'wb') as f:
            pickle.dump(d, f)



# Build metadata.csv

### Choose tags

In [ ]:
high_risk_tags = ['ir_flag', 'active_rescue','suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk']

In [ ]:
tags_alphabetical = ['3rd_party',
 'abuse_emotional',
 'abuse_physical',
 'abuse_sexual',
 # 'abuse_unspecified',
 'anxiety',
 'bereavement',
 'bully',
 'covid_19',
 'depressed',
 'domestic_violence',
 'eating',
 'election',
 'family',
 'financial',
 'friend',
 'gender',
 'homeless',
 'homicide',
 'immigration',
 'isolated',
 'medical',
 'medication',
 'mental',
 'military',
 'none',
 'other',
 'out_of_country',
 'prank',
 'psychiatric_hospitalization',
 'racism',
 'relationship',
 'school',
 'self_harm',
 'sexual_assault',
 'stress',
 'substance',
 'suicide',
 'testing',
 'therapist_psychiatrist',
 'work']

In [ ]:

cols_to_merge_d = {
     'work':'work_financial', #only exists pre 2018
     'financial':'work_financial', #only exists pre 2018
    'sexual_assault':'abuse_sexual', #changed names
    'anxiety':'anxiety_stress', #was an option for counselors "Anxiety/Stress"
    'stress':'anxiety_stress', 

     # 'abuse_phyiscal':'abuse_physical',
     # 'abuse_physical':'abuse_physical',
     # 'physical_abuse':'abuse_physical',
     # 'abuse_sexual':'abuse_sexual',
     # 'abuse_unspecified':'abuse',
     # 'domestic_violence':'domestic_violence',
     # 'sexual_abuse':'abuse_sexual',

     # 'alcohol': 'substance',
     # 'relationship_violence':'relationship',
     # 'family':'relationship',
     # 'friend':'relationship',
     'hunger':'eating_body_image',
    'eating':'eating_body_image',
    
     'gender':'gender_sexual_identity'
    }

def merge_cols(df, cols_to_merge_d):
    
    for key, value in cols_to_merge_d.items():
        if not value in df.columns:
            df[value] = 0
        try:
            df[value] += df[key]
        except:
            pass
    for key in cols_to_merge_d.keys():
        try: df = df.drop(key,axis=1)
        except:
            pass
    return df

In [ ]:
tags_high_freq = [
    'depressed',
 'relationship',
 'anxiety',
 'suicide',
 'none',
 'isolated',
 # 'other',
 'self_harm',
 'bereavement',
 # '3rd_party',
 'eating',
 'abuse_emotional',
 'family',
 # 'testing',
 'bully',
 'substance',
 'abuse_sexual',
 'covid_19',
 'gender',
 'abuse_physical',
 'stress',
 'friend',
 'school',
 # 'abuse_unspecified',
 # 'prank',
 # 'mental',
 'work',
 'financial',
 # 'medical',
 'therapist_psychiatrist',
 'medication',
 'sexual_assault',
 # 'homeless',
 # 'domestic_violence',#low frequency
 # 'psychiatric_hospitalization', #4500 in 2017
 # 'racism', #3k across the years
 # 'election', #2.5k across the years
 # 'military',
 # 'homicide',
 # 'immigration',
 # 'out_of_country'
]




# create a dataset for concurrent
concurrent_variables = np.unique(high_risk_tags+tags_high_freq+list(cols_to_merge_d.values()))
concurrent_variables        


In [ ]:
# 'train10_train' was done 

os.listdir(input_dir)


    

In [ ]:
%%time
# create DF from convo IDs

run_this = False


if run_this:

	for set_name in ['train10_train', 'train10_val', 'train10_test']:
	# for set_name in ['train10_train']:

		with open(input_dir+f'train_test_split_ids/{set_name}.pkl', 'rb') as f:
			d = pickle.load(f)

		df_set = []

		for year in [2017,2018,2019,2022]:
			d_year_ids = d.get(year)
			df_year = pd.read_parquet(input_dir+f'raw_data/tags_risk_{year}_dummyvars.gzip', engine='pyarrow')        
			df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)]
			df_set.append(df_year_set)

		df_set = pd.concat(df_set).reset_index(drop=True)    
		os.makedirs(input_dir+f'datasets', exist_ok=True)
		df_set.to_csv(input_dir+f'datasets/{set_name}_metadata.csv')

# metadata_csv to train10_train_concurrent_metadata (balanced 3k each crisis)


In [ ]:
# Load
df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
df_set.shape

In [ ]:
df_set.columns

In [ ]:
concurrent_variables_in_df = [n for n in variables_of_interest if n in df_set.columns]

counts = df_set[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

In [ ]:
display((df_set[concurrent_variables_in_df].isna().sum()/df_set.shape[0]).round(2))

In [ ]:
df_set.shape

In [ ]:
df_set2 = merge_cols(df_set, cols_to_merge_d)

In [ ]:
set(df_set2.columns) - set(df_set.columns)

In [ ]:
concurrent_variables_in_df = [n for n in concurrent_variables if n in df_set2.columns]


In [ ]:
(df_set2[concurrent_variables_in_df].isna().sum()/df_set2.shape[0]).round(2)

In [ ]:
concurrent_variables_in_df

In [ ]:
counts = df_set2[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

In [ ]:
counts.sort_values('count')[::-1]['construct'].values

In [ ]:
concurrent_analysis_vars = concurrent_variables_in_df.copy()
drop_cols = ['work_financial', 'covid_19', 'medication', 'active_rescue', 'ir_flag']
[concurrent_analysis_vars.remove(n) for n in drop_cols]
concurrent_analysis_vars

In [ ]:
drop_cols = ['work_financial', 'covid_19', 'medication', 'active_rescue', 'ir_flag']
df_set = df_set.drop(drop_cols, axis=1)

In [ ]:
print(len(concurrent_analysis_vars))

In [ ]:
# Balance: Take n for each construct and concat. This is useful for one vs all classification. 
concurrent_df_n = 3000

concurrent_df = []
for variable in concurrent_analysis_vars:
    df_set2_variable_i = df_set2[df_set2[variable]==1].sample(n = concurrent_df_n)
    concurrent_df.append(df_set2_variable_i)
    
concurrent_df = pd.concat(concurrent_df).reset_index(drop=True)

In [ ]:
counts = concurrent_df[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

In [ ]:
concurrent_df = concurrent_df.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)




### Save DF 

In [ ]:
input_dir

In [ ]:
concurrent_df = concurrent_df.drop_duplicates('conversation_id')
concurrent_df.to_csv(input_dir+'../datasets/train10_train_concurrent_metadata.csv')

In [ ]:
# Load
df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
df_set.shape

# train 10 with risk ladders




In [ ]:
def get_true_risk_3(row):
	if (row['3rd_party'] ==1 or row['testing'] == 1 or row['prank'] == 1):
		return -1
	elif (row['active_rescue'] > 0 or row['ir_flag'] > 0 or row['timeframe'] > 0):
		return 3 # high risk
	elif ('suicidal_desire' in row['suicidality'] or 'suicidal_intent' in row['suicidality'] or 'suicidal_capability' in row['suicidality']  or row['suicide']>0 or row['self_harm']>0): 
		return 2 # medium risk
	else:
		return 1 # normal risk
	

def get_true_risk_8(row):
	if (row['3rd_party'] ==1 or row['testing'] == 1 or row['prank'] == 1):
		return -1
	elif (row['active_rescue'] > 0):
		return 8 # active rescue
	
	elif (row['ir_flag'] > 0):
		return 7 # high risk
	
	elif (row['timeframe'] > 0):
		return 6 # high risk
	
	elif (row['suicidal_capability'] > 0):
		return 5 # high risk
	
	elif (row['suicidal_intent']>0):
		return 4
	elif row['self_harm']>0:
		return 3

	elif (row['suicidal_desire']>0 or row['suicide']>0):
		return 2
	else: 
		return 1
		


In [ ]:
input_dir = '/Users/danielmlow/data/ctl/input/'

In [ ]:
# OUt of active rescues how many had timeframe tagged
metadata_df_i = pd.read_csv(input_dir+f'datasets/train10_test_metadata.csv', index_col=0)

display(metadata_df_i[metadata_df_i['active_rescue']==1]['timeframe'].value_counts())
display(metadata_df_i[metadata_df_i['active_rescue']==1]['ir_flag'].value_counts())
display(metadata_df_i[metadata_df_i['ir_flag']==1]['timeframe'].value_counts())
display(metadata_df_i[metadata_df_i['suicidal_intent']==1]['suicidal_desire'].value_counts())
display(metadata_df_i[metadata_df_i['suicide']==1]['suicidal_intent'].value_counts())
display(metadata_df_i[metadata_df_i['suicidal_intent']==1]['suicidal_capability'].value_counts())

In [ ]:
metadata_df_i.columns

In [ ]:
tags_of_interest = ['abuse_emotional', 'abuse_physical', 'abuse_sexual',
        'anxiety', 'stress', 'anxiety_stress', 'bereavement',
       'bully', 'covid_19', 'depressed', 'eating', 'eating_body_image',
       'family', 'financial', 'friend', 'gender',
       'gender_sexual_identity', 'isolated', 'medication',
       'none', 'relationship', 'school',  'sexual_assault',
       'stress', 'substance', 
         'therapist_psychiatrist',
        'work', 'work_financial', 
		'perceived_risk','self_harm','suicide','suicidal_desire', 'suicidal_intent','suicidal_capability','timeframe', 'ir_flag', 'active_rescue',
		'suicide_ladder_a', 'suicide_ladder_b', 'suicide_ladder_c']




tags_of_interest_in_df = [n for n in tags_of_interest if n in metadata_df_i.columns]

metadata_df_i[tags_of_interest_in_df].sum()

In [ ]:
metadata_df_i = pd.read_csv(input_dir+f'datasets/train10_test_metadata.csv', index_col=0)

metadata_df_i = metadata_df_i.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)


metadata_df_i['suicide_ladder_a'] = [get_true_risk_a(row) for index, row in metadata_df_i.iterrows()]
metadata_df_i['suicide_ladder_b'] = [get_true_risk_b(row) for index, row in metadata_df_i.iterrows()]
metadata_df_i['suicide_ladder_c'] = [get_true_risk_c(row) for index, row in metadata_df_i.iterrows()]

In [ ]:
display(metadata_df_i['suicide_ladder_a'].value_counts())
display(metadata_df_i['suicide_ladder_b'].value_counts())
display(metadata_df_i['suicide_ladder_c'].value_counts())


In [ ]:
metadata_df_i

In [ ]:
metadata_df_i

In [ ]:
# How many suicide risk do we have?

input_dir = './../../../data/ctl/input/'

run_this = False

if run_this:

	for set_name in ['train10_train', 'train10_val', 'train10_test']:
		print(input_dir+f'datasets/{set_name}_metadata.csv')
		metadata_df_i = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
		
		print(metadata_df_i.shape)
		
		metadata_df_i = metadata_df_i.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)
		
		
		metadata_df_i['suicide_ladder_3'] = [get_true_risk_3(row) for index, row in metadata_df_i.iterrows()]
		metadata_df_i['suicide_ladder_8'] = [get_true_risk_8(row) for index, row in metadata_df_i.iterrows()]

		metadata_df_i.to_csv(input_dir+f'datasets/{set_name}_metadata.csv')

		# or subsample:
		# if set_name == 'train10_train':
		# 	metadata_df_i = metadata_df_i.sample(frac= 0.3, random_state = 123)
			
			
		# else:
		# 	metadata_df_i = metadata_df_i.sample(frac= 0.15, random_state = 123)
		# 	metadata_df_i.to_csv(input_dir+f'datasets/{set_name}_15perc_metadata.csv')


In [ ]:
set_name

In [ ]:
display(metadata_df_i['suicide_ladder_a'].value_counts())
display(metadata_df_i['suicide_ladder_b'].value_counts())
display(metadata_df_i['suicide_ladder_c'].value_counts())


# Find messages from metadata.csv files

In [ ]:
import pandas as pd
import pickle


set_names = ['train10_train' ,'train10_val','train10_test']

In [ ]:
# How many suicide risk do we have?


for set_name in set_names:
	metadata_df_i = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
	tags_of_interest_in_df = [n for n in tags_of_interest if n in metadata_df_i.columns]

	counts = metadata_df_i[tags_of_interest_in_df].sum()
	counts = pd.DataFrame(counts).reset_index()
	counts.columns = ['construct', 'count']
	display(counts.sort_values('count')[::-1])

	



In [ ]:
display(metadata_df_i['tags_sorted'])
display(metadata_df_i[['tags_sorted', 'suicidality_sorted', 'perceived_risk', 'suicide', 'ir_flag', 'active_rescue']])
display(metadata_df_i[['ir_flag', 'active_rescue','active_rescue2' ]].sum())
display(metadata_df_i[['perceived_risk' ]].value_counts())

In [ ]:
# # Load
# df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
# df_set.shape

# # concurrent_df  = pd.read_csv(input_dir+'datasets/train10_train_concurrent_metadata.csv', index_col = 0)



In [ ]:
conversation_cols = [
    'conversation_id',
    'conversation_start_time_utc',
 'conversation_end_time_utc',
 'post_conversation_survey_end_time',
 'priority',
 'priority_predicted',
 'presenting_concern_message_id',
 'is_risk_assessed',
 'keyword',
 'num_wait_time_notifications',
 'language']

conversation_cols_to_drop = [e for e in conversation_cols if e not in ['conversation_id',
 'conversation_start_time_utc']]

In [ ]:
# metadata_df_ids = metadata_df_i['conversation_id'].values
# metadata_df_ids

In [ ]:
set_name = 'train10_train'


    




    

In [ ]:
d.keys()

In [ ]:



%%time
# 8min
# create training DF from convo IDs

run_this = True
if run_this:

	for set_name in set_names: 
		metadata_df = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)

		metadata_df_ids = metadata_df['conversation_id'].values
		with open(input_dir+f'train_test_split_ids/{set_name}.pkl', 'rb') as f:
			d = pickle.load(f)



		df_set = []
		df_unique_per_conversation_all = []
		years = [2017,2018,2019,2022]
		# if toy: 
		#     years = years[:3]

		for year in years:
			print(year, '=======')
			d_year_ids = d.get(year) # get list of conversation_ids in this year
			
			
			
			df_year = []
			for month in range(1,13):
				print(month)
				month = str(month).zfill(2)
				df_year_month_i = pd.read_parquet(input_dir+f'raw_data/messages_{year}-{month}.gzip', engine='pyarrow')       
				df_year.append(df_year_month_i)
			df_year = pd.concat(df_year).reset_index(drop=True) # all messages for that year
		
			# else:
			#     df_year = pd.read_parquet(input_dir+f'raw_data/messages_{year}.gzip', engine='pyarrow') 
				
			# df_set.to_parquet(input_dir+f'datasets/train10_train_concurrent_messages.gzip', compression = 'gzip', engine='pyarrow')        
			
			# choose conversation that are in metadata DF of interest
			
			df_year_set = df_year[df_year['conversation_id'].isin(metadata_df_ids)] # NEW
			# df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)] # OLD
			
			# these messages_df cols are unique within a conversation, so add to metadata_df and remove from messages_df to save space
			df_year_unique_per_conversation = df_year_set[conversation_cols] 
			df_year_unique_per_conversation = df_year_unique_per_conversation.drop_duplicates()
			
			
			# normalize formatting
			# concurrent_df = concurrent_df.astype(str).reset_index(drop=True)
			# df_year_unique_per_conversation = df_year_unique_per_conversation.astype(str).reset_index(drop=True)
			df_unique_per_conversation_all.append(df_year_unique_per_conversation)
			# df_year_unique_per_conversation['conversation_start_time_utc'] = pd.to_datetime(df_year_unique_per_conversation['conversation_start_time_utc'])
			# df_year_unique_per_conversation['conversation_end_time_utc'] = pd.to_datetime(df_year_unique_per_conversation['conversation_end_time_utc'])
			# df_year_unique_per_conversation['post_conversation_survey_end_time'] = pd.to_datetime(df_year_unique_per_conversation['post_conversation_survey_end_time'])
			
			
			
			# # todo merge with concurrent
			# if year == years[0]:            
			#     # concurrent_df['conversation_start_time_utc'] = pd.to_datetime(concurrent_df['conversation_start_time_utc'])
			#     concurrent_df = concurrent_df.merge(df_year_unique_per_conversation, on=['conversation_id', 'conversation_start_time_utc'] , how = 'left')    
			# else:           
			#     concurrent_df = concurrent_df.merge(df_year_unique_per_conversation, on=conversation_cols, how = 'left')
			
			
			df_year_set = df_year_set.drop(conversation_cols_to_drop, axis=1)
			
			# TODO: uncomment line below
			# df_year_set.to_parquet(input_dir+f'datasets/train10_train_concurrent_messages_{year}.gzip', compression = 'gzip', engine='pyarrow')        
			
			df_set.append(df_year_set)

		# concurrent_df.to_csv(input_dir+'datasets/train10_train_concurrent_metadata2.csv')
		df_set = pd.concat(df_set).reset_index(drop=True)        
		df_set.to_parquet(input_dir+f'datasets/{set_name}_messages.gzip', compression = 'gzip', engine='pyarrow')        
		df_year_texter = df_set[df_set['interaction']=='texter']
		df_year_texter.to_parquet(input_dir+f'datasets/{set_name}_messages_texter.gzip', compression = 'gzip', engine='pyarrow')        
		df_year_counselor = df_set[df_set['interaction']=='counselor']
		df_year_counselor.to_parquet(input_dir+f'datasets/{set_name}_messages_counselor.gzip', compression = 'gzip', engine='pyarrow')        
		
		df_unique_per_conversation_all = pd.concat(df_unique_per_conversation_all).reset_index(drop=True)        
		df_unique_per_conversation_all.to_parquet(input_dir+f'datasets/{set_name}_messages_columns_unique_to_convo.gzip', compression = 'gzip', engine='pyarrow')        
		
		
		

# Concat messages
 

In [ ]:
set_names = ['train10_train' ,'train10_val','train10_test']

dataset_dir = '/Users/danielmlow/data/ctl/input/datasets/'

# sub_dir = 'train10_subset_30', add {sub_dir}/

# Text 
train_texter = pd.read_parquet(dataset_dir + f'{set_names[0]}_messages_texter.gzip', engine='pyarrow')
val_texter = pd.read_parquet(dataset_dir + f'{set_names[1]}_messages_texter.gzip', engine='pyarrow')
test_texter = pd.read_parquet(dataset_dir + f'{set_names[2]}_messages_texter.gzip', engine='pyarrow')

# Both
train_both = pd.read_parquet(dataset_dir + f'{set_names[0]}_messages.gzip', engine='pyarrow')
val_both = pd.read_parquet(dataset_dir + f'{set_names[1]}_messages.gzip', engine='pyarrow')
test_both = pd.read_parquet(dataset_dir + f'{set_names[2]}_messages.gzip', engine='pyarrow')

# Metadata (i.e., target variables)
train_metadata = pd.read_csv(dataset_dir + f'{set_names[0]}_metadata.csv')
val_metadata = pd.read_csv(dataset_dir+ f'{set_names[1]}_metadata.csv')
test_metadata = pd.read_csv(dataset_dir + f'{set_names[2]}_metadata.csv')

In [ ]:
test_both_i = test_both.copy()

column = 'message'
test_both_i['user_message'] =test_both_i['interaction']+': '+test_both_i[column]
test_both_i['user_message']
X_i = '\n'.join(test_both_i['user_message'].tolist()) # messages of 1 convo
print(X_i[:1000])

In [ ]:
def merge_concatenated_messages(metadata_df, messages_df,merge_on = 'conversation_id',  column = 'message', new_column = 'text_texter'):
	convo_ids = []
	X = []
	for i in metadata_df[merge_on].unique():
		messages_df_i = messages_df[messages_df[merge_on]==i]
		if new_column =='text_texter':
			messages_convo_i = [n.strip(' ') if n.endswith(('.', ',', ']', ')', '!','?', '>')) else n.strip(' ')+'.' for n in messages_df_i[column].tolist() ]
			X_i = ' '.join(messages_convo_i) # messages of 1 convo
		elif new_column =='text_both':
			# use user column (texter or counselor) to create a single string with counselor: <message> \n texter: <message> depending on user column
			messages_df_i['user_message'] = messages_df_i['interaction']+': '+messages_df_i[column]
			X_i = '\n'.join(messages_df_i['user_message'].tolist()) # messages of 1 convo
			
		
		# X_i = X_i.replace('
		X.append(X_i)
		convo_ids.append(i)

	df_text = pd.DataFrame({merge_on: convo_ids,
							new_column:X,
						})


	metadata_df = metadata_df.merge(df_text, on=merge_on, how='left')
	return metadata_df

In [ ]:
test_both_i = test_both[['conversation_id', 'message_timestamp_utc', 'actor_id'] + ['interaction', 'message']]
print(test_both_i.shape)
# Concat messages
test_both_i['message_with_interaction'] = test_both_i['interaction']+': '+test_both_i['message']
message_with_interaction_concatenated = test_both_i.groupby('conversation_id')['message_with_interaction'].apply('\n'.join).reset_index()


# Last message
last_message_timestamp = test_both_i.groupby('conversation_id')['message_timestamp_utc'].last().reset_index()
# Actor ID
test_both_i_texter = test_both_i[test_both_i['interaction']=='texter']
actor_id = test_both_i_texter.groupby('conversation_id')['actor_id'].last().reset_index()

test_concat = message_with_interaction_concatenated.merge(last_message_timestamp, on='conversation_id').merge(actor_id, on='conversation_id')


print(test_concat.shape)


In [ ]:
set_names

In [ ]:
for set_name, messages_texter_i, messages_both_i, metadata_i in zip(set_names, [train_texter, val_texter, test_texter], [train_both, val_both, test_both], [train_metadata, val_metadata, test_metadata]):
	print(set_name)
	convo_ids = messages_texter_i['conversation_id'].unique().tolist()[:100]

	# subsample
	subsample = False
	if subsample:
		messages_texter_i = messages_texter_i[messages_texter_i['conversation_id'].isin(convo_ids)]
		messages_both_i = messages_both_i[messages_both_i['conversation_id'].isin(convo_ids)]
		metadata_i = metadata_i[metadata_i['conversation_id'].isin(convo_ids)]
	
	# Both
	# ====================================
	messages_both_i = messages_both_i[['conversation_id', 'message_timestamp_utc', 'actor_id'] + ['interaction', 'message']]
	print(messages_both_i.shape)
	# Concat messages
	messages_both_i['message_with_interaction'] = messages_both_i['interaction']+': '+messages_both_i['message']
	message_with_interaction_concatenated = messages_both_i.groupby('conversation_id')['message_with_interaction'].apply('\n'.join).reset_index()
	# Last message
	last_message_timestamp = messages_both_i.groupby('conversation_id')['message_timestamp_utc'].last().reset_index()
	# Actor ID
	messages_both_i_texter = messages_both_i[messages_both_i['interaction']=='texter']
	actor_id = messages_both_i_texter.groupby('conversation_id')['actor_id'].last().reset_index()
	messages_both_i_concat = message_with_interaction_concatenated.merge(last_message_timestamp, on='conversation_id').merge(actor_id, on='conversation_id')
	print(messages_both_i_concat.shape)
	metadata_i = metadata_i.merge(messages_both_i_concat, on='conversation_id', how='left')
	print(metadata_i.shape)
	# metadata_i.to_parquet(input_dir+f'datasets/{set_name}_metadata_messages_both.gzip', compression = 'gzip', engine='pyarrow')

	# Texter
	# ====================================
	messages_texter_i = messages_texter_i[['conversation_id', 'message_timestamp_utc', 'actor_id'] + ['interaction', 'message']]
	print(messages_texter_i.shape)
	# Concat messages
	message_with_interaction_concatenated = messages_texter_i.groupby('conversation_id')['message'].apply('\n'.join).reset_index()
	
	metadata_i = metadata_i.merge(message_with_interaction_concatenated, on='conversation_id', how='left')
	print(metadata_i.shape)
	# metadata_i['message_joined'] = metadata_i['message'].replace('\n', '.')
	# [n.strip(' ') if n.endswith(('.', ',', ']', ')', '!','?', '>')) else n.strip(' ')+'.' for n in messages_df_i[column].tolist() ]
	metadata_i.to_parquet(input_dir+f'datasets/{set_name}_metadata_messages.gzip', compression = 'gzip', engine='pyarrow')



# # Last message
	# last_message_timestamp = messages_texter_i.groupby('conversation_id')['message_timestamp_utc'].last().reset_index()
	# # Actor ID
	# actor_id = messages_texter_i.groupby('conversation_id')['actor_id'].last().reset_index()
	# messages_texter_i_concat = message_with_interaction_concatenated.merge(last_message_timestamp, on='conversation_id').merge(actor_id, on='conversation_id')
	# print(messages_texter_i_concat.shape)

In [ ]:
metadata_i

In [ ]:
'body_image' in metadata_i.columns

In [ ]:
set_names


In [ ]:
ctl_tags13 = ['self_harm',
 'suicide',
 'bully',
 'abuse_physical',
 'abuse_sexual',
 'relationship',
 'bereavement',
 'isolated',
 'anxiety', #anxiety_stress
 'depressed',
 'gender', #gender_sexual_identity
 'eating', # eating_body_image
 'substance']



for set_name in set_names:
	metadata_messages_i = pd.read_parquet(dataset_dir + f'{set_name}_metadata_messages.gzip', engine='pyarrow')
	display(metadata_messages_i[ctl_tags13].sum())
	


	
# 	# 


	

In [ ]:
minimum_train = 3077
minimum_val = 410
minimum_test = 376






In [ ]:
metadata_i_both

In [ ]:
display(metadata_i_texter)
display(metadata_i_both)

In [ ]:
# for set_name in set_names:
# 	df_set = pd.read_parquet(input_dir+f'datasets/{set_name}_messages.gzip', engine='pyarrow')        
# 	break





In [ ]:
# df_set

In [ ]:
cols_not_added = [n for n in df_year_month_i.columns if n not in df_set.columns]
cols_not_added


In [ ]:
for col in df_year_month_i[cols_not_added].columns:
	print(col)
	display(df_year_month_i[cols_not_added][col].value_counts(normalize=True).iloc[:10])


In [ ]:
metadata_df

# Build tokenized DF

In [ ]:
import numpy as np

In [ ]:
# TODO: merge active_si and passive_si, change name of self-injury


constructs_to_measure = ['self-injury',
 'active_si',
 'passive_si',
 'bully',
 'abuse_physical',
 'abuse_sexual',
 'relationships',
 'grief_bereavement',
 'loneliness_isolated',
 'anxiety',
 'depressed_mood',
 'gender_sexual_identity',
 'eating_disorder',
 'substance_use']



# values = ['']*len(constructs_to_measure)
# ctl_tags = dict(zip(constructs_to_measure, values))
ctl_tags_d = {'self-injury': 'self_harm',
 'active_si': 'suicide',
 'passive_si': 'suicide',
 'bully': 'bully',
 'abuse_physical': 'abuse_physical',
 'abuse_sexual': 'abuse_sexual',
 'relationships': 'relationship',
 'grief_bereavement': 'bereavement',
 'loneliness_isolated': 'isolated',
 'anxiety': 'anxiety_stress',
 'depressed_mood': 'depressed',
 'gender_sexual_identity': 'gender_sexual_identity',
 'eating_disorder': 'eating_body_image',
 'substance_use': 'substance'}

ctl_tags = np.unique(list(ctl_tags_d.values())).tolist()
ctl_tags

In [ ]:
run_this = False
if run_this:

    df_messages = pd.read_parquet(input_dir+f'datasets/train10_train_concurrent_messages_texter.gzip', engine='pyarrow')        
    df_metadata = pd.read_csv(input_dir+f'datasets/train10_train_concurrent_metadata.csv', index_col = 0)
    
    
    df_metadata = df_metadata.drop_duplicates('conversation_id').reset_index(drop=True)
    print(df_metadata.shape, df_messages.shape)

    # Create a subset of metadata that has at least n_samples of a given construct
    n_samples = 100

    df_metadata_subsample = []

    for n in ctl_tags:
        df_metadata_tag = df_metadata[df_metadata[n]==1].sample(n=n_samples,random_state=123)
        df_metadata_subsample.append(df_metadata_tag)

    df_metadata_subsample = pd.concat(df_metadata_subsample).reset_index(drop=True)
    print(df_metadata_subsample.shape)
    df_metadata_subsample_tags = df_metadata_subsample[['conversation_id']+ctl_tags]
    df_metadata_subsample_tags = df_metadata_subsample_tags.drop_duplicates().reset_index(drop=True)
    # print(df_metadata_subsample_tags.shape)
    # df_metadata_subsample_tags.to_csv(input_dir+"multioutput_"+training_lexicon+'.csv')
    
    # Find messages for the subsample
    df_messages_subsample = df_messages[df_messages['conversation_id'].isin(df_metadata_subsample_tags['conversation_id'])]
    df_messages_subsample = df_messages_subsample.drop_duplicates('message_id').reset_index(drop=True)
    print(df_metadata_subsample_tags.shape)

    # df_test_multioutput_subsample = df_messages_subsample.merge(df_metadata_subsample_tags,on='conversation_id', how='left')
    # print(df_test_multioutput_subsample.shape)
    # df_test_multioutput_subsample['text'] = df_test_multioutput_subsample['message'] 

    # # add label column for multi-output training
    # # df_test_multioutput_subsample = pd.read_csv(input_dir+"multioutput_ctl_train10_1266_"+training_lexicon+'.csv', index_col = 0)
    # df_test_multioutput_subsample['labels'] = [np.array(n) for n in df_test_multioutput_subsample[ctl_tags].values]
    # df_test_multioutput_subsample = df_test_multioutput_subsample.drop_duplicates('message_id')
    # df_test_multioutput_subsample.to_csv(input_dir+"multioutput_ctl_train10_1266_"+training_lexicon+'.csv')


In [ ]:
'SURVEY' in top_texter_messages

In [ ]:
counts = pd.DataFrame(df_messages_subsample['message'].value_counts())

# after reviewing, remove top 100 messages, which are a lot of keywords to log in. 
top_texter_messages = counts.iloc[:100].index.tolist()
top_texter_messages.remove('I want to die')
top_texter_messages.remove('I need help')
print(df_messages_subsample.shape)
df_messages_subsample = df_messages_subsample[~df_messages_subsample['message'].isin(top_texter_messages)]
print(df_messages_subsample.shape)

In [ ]:
df_messages_subsample

In [ ]:
df = df_messages_subsample[['conversation_id','message']].groupby(['conversation_id'])['message'].transform(lambda x: '. '.join(x))


df

In [ ]:
convo_messages = {}

for convo_id in df_messages_subsample['conversation_id'].unique():
    df_messages_subsample_convo_i = df_messages_subsample[df_messages_subsample['conversation_id']== convo_id]
    messages = df_messages_subsample_convo_i['message'].tolist()
    # [print(n)  for n in messages if 'scrubbed' in n]
    messages = '. '.join([n.strip().strip('.').replace('[scrubbed]', 'Noah').replace('{{PHONE}}', '555-857-2939') for n in messages]) #unisex name
    convo_messages[convo_id] = messages

In [ ]:
df_metadata_subsample['message'] = df_metadata_subsample['conversation_id'].map(convo_messages)

df_metadata_subsample

In [ ]:
df_metadata_subsample.columns

In [ ]:
df_metadata_subsample.to_csv(
    input_dir+f'datasets/train10_train_concurrent_metadata_100perconstruct_with_messages.csv')


In [ ]:
input_dir+f'datasets/train10_train_concurrent_metadata_100perconstruct_with_messages.csv'

### Build small dataset

In [ ]:
import pandas as pd

In [ ]:
input_dir = './../../../data/ctl/input/'

In [ ]:
conversation_cols = [
    'conversation_id',
    'conversation_start_time_utc',
 'conversation_end_time_utc',
 'post_conversation_survey_end_time',
 'priority',
 'priority_predicted',
 'presenting_concern_message_id',
 'is_risk_assessed',
 'keyword',
 'num_wait_time_notifications',
 'language']

conversation_cols_to_drop = [e for e in conversation_cols if e not in ['conversation_id',
 'conversation_start_time_utc']]

In [ ]:
input_dir

In [ ]:
df_metadata = pd.read_csv(input_dir+f'datasets/train10_train_concurrent_metadata.csv', index_col = 0)

df_messages = pd.read_parquet(input_dir+f'datasets/train10_train_concurrent_messages_texter.gzip', engine='pyarrow')        

In [ ]:
for message in message:
    

In [ ]:
%%time 
year = 2017
df_year = pd.read_parquet(input_dir+f'raw_data/messages_{year}.gzip', engine='pyarrow')        

In [ ]:
df_year.head()

In [ ]:
for n in cols_to_drop:
    display(df_year[n].value_counts())

In [ ]:
concurrent_df.columns

In [ ]:
%%time
# create training DF from convo IDs

run_this = True


if run_this:

    df_set = []

    for year in [2017,2018,2019,2022]:
        print(year)
        d_year_ids = d.get(year)
        a
        # df_year.to_parquet(input_dir+f'messages_{year}.gzip', compression = 'gzip', engine='pyarrow')        

        df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)]
        df_set.append(df_year_set)


    # df_set.to_csv(input_dir+f'../datasets/train10_train_messages.csv')

In [ ]:
df_set = pd.concat(df_set)

In [ ]:
    df_set = pd.concat(df_set)
    df_set = df_set.reset_index(drop=True)    
    os.makedirs(input_dir+f'../datasets', exist_ok=True)
    df_set.to_parquet(input_dir+f'../datasets/train10_train_messages.gzip', compression = 'gzip', engine='pyarrow')        

In [ ]:
df_set

In [ ]:
df_set

In [ ]:
df_set

### Load

In [ ]:
input_dir = './../../../data/ctl/input/'

In [ ]:
metadata_df = pd.read_csv(input_dir+'datasets/train10_train_metadata.csv', index_col = 0)

In [ ]:

# messages_df = pd.read_csv(input_dir+'../datasets/train10_train_messages.csv', index_col = 0, lineterminator='\n')


In [ ]:
messages_df

In [ ]:
messages_df['conversation_id'].unique().shape

In [ ]:
metadata_df.shape

In [ ]:
concurrent_df[concurrent_df['isolated']==1]['conversation_id'].values

# OLD

In [ ]:
metadata_i.columns

In [ ]:
binary_columns = ['ir_flag', 'active_rescue','abuse', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anger', 'anxiety', 'bereavement',
       'bully', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'housing', 'hunger', 'isolated',
       'lgbtq', 'medical', 'military', 'none', 'other', 'prank', 'pregnancy',
       'relationship', 'self_harm', 'sexual_abuse', 'stress_anxiety',
       'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe']

In [ ]:
messages_i['actor_id'].value_counts()

In [ ]:
metadata_i['active_rescue2'].shape

In [ ]:

(metadata_i['ir_flag']==metadata_i['active_rescue2']).sum()/metadata_i.shape[0]

In [ ]:
metadata_i[metadata_i['active_rescue']==1]

In [ ]:
year_month = '2022-12'
messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
for i in [8030479,8030508, 8030786, 8031145]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

In [ ]:
tag_cols = ['abuse', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anger', 'anxiety', 'bereavement',
       'bully', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'housing', 'hunger', 'isolated',
       'lgbtq', 'medical', 'military', 'none', 'other', 'prank', 'pregnancy',
       'relationship', 'self_harm', 'sexual_abuse', 'stress_anxiety',
       'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk']

In [ ]:
print(metadata_i.shape)
print(metadata_i[metadata_i['engaged']==0].shape)

In [ ]:
metadata_i[metadata_i['engaged']==0][tag_cols].sum()

In [ ]:
metadata_i[(metadata_i['engaged']==0) & (metadata_i['relationship']==1)]

In [ ]:
for i in [697402,697657, 698727, 683578, 700002]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

In [ ]:
for year_month in year_months:
    messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        
    break

In [ ]:
messages_i.sum()

In [ ]:
for i in [700001,700005, 700009]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

In [ ]:
year_month = '2016-04'
messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
year_month = '2017-01'
messages_17 = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
messages_17

In [ ]:
cols = ['conversation_id', 'actor_id']

In [ ]:
%%time
convo_actor_ids = {}


for convo_id in messages_i['conversation_id'].values:
    convo_actor_ids[convo_id] = len(messages_i[messages_i['conversation_id'] == convo_id]['actor_id'].unique())
    
    
    
    


In [ ]:
pd.DataFrame(convo_actor_ids)

In [ ]:


messages_i.groupby(cols)['actor_id'].count()

In [ ]:
print(messages_i[messages_i['conversation_id']==360679]['id'].unique())
messages_i[messages_i['conversation_id']==360679]['id']

In [ ]:
messages_i[messages_i['conversation_id']==360679]['message'].values

In [ ]:
messages_i

In [ ]:
messages_17

In [ ]:
year_month = '2017-01'
messages_17 = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
messages_17

In [ ]:
metadata_i[metadata_i['depressed']==1]